<a href="https://colab.research.google.com/github/Swetha192004/real-estate-management/blob/main/Copy_of_Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/narumiruna/efficientnet-pytorch.git


Cloning into 'efficientnet-pytorch'...
remote: Enumerating objects: 876, done.
remote: Counting objects: 100% (304/304), done.
remote: Compressing objects: 100% (181/181), done.
remote: Total 876 (delta 168), reused 208 (delta 121), pack-reused 572 (from 1)
Receiving objects: 100% (876/876), 293.73 KiB | 6.53 MiB/s, done.
Resolving deltas: 100% (479/479), done.


In [ ]:
!pip install torch torchvision

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json



Saving 0001.jpg to 0001.jpg


{'0001.jpg': b'\xff\xd8\xff\xe1\x8faExif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x0c\x01\x00\x00\x03\x00\x00\x00\x01\x0b\xb8\x00\x00\x01\x01\x00\x03\x00\x00\x00\x01\x0b\xb8\x00\x00\x01\x0f\x00\x02\x00\x00\x00\x08\x00\x00\x00\x9e\x01\x10\x00\x02\x00\x00\x00\x07\x00\x00\x00\xa6\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00\xae\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00\xb6\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\xbe\x02\x13\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\xd2\x88%\x00\x04\x00\x00\x00\x01\x00\x00\x036\x00\x00\x04\x10OnePlus\x00GM1901\x00\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x012021:02:04 18:06:05\x00\x00 \x82\x9a\x00\x05\x00\x00\x00\x01\x00\x00\x02X\x82\x9d\x00\x05\x00\x00\x00\x01\x00\x00\x02`\x88"\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x88\'\x00\x03\x00\x00\x00\x01\t\xc4\x00\x00\x90\x00\x00\x07\x00\x00\x00\x040220\x90\x03\x00\x02\x

In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets download -d uciml/zoo-animal-classification -p datasets --unzip

Dataset URL: https://www.kaggle.com/datasets/uciml/zoo-animal-classification
License(s): DbCL-1.0
  0% 0.00/1.85k [00:00<?, ?B/s]
100% 1.85k/1.85k [00:00<00:00, 4.94MB/s]


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim

# Load the dataset
data = pd.read_csv('datasets/zoo.csv')

# Separate features and target
X = data.drop(columns=['animal_name', 'class_type'])
y = data['class_type']

# Encode the target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
class ZooAnimalDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create dataset instances
train_dataset = ZooAnimalDataset(X_train_scaled, y_train)
test_dataset = ZooAnimalDataset(X_test_scaled, y_test)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
class MLP(nn.Module):
    def __init__(self, input_dim, num_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, num_classes)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize the model, loss function, and optimizer
input_dim = X_train_scaled.shape[1]
num_classes = len(label_encoder.classes_)
model = MLP(input_dim, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader)}')

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for features, labels in test_loader:
            outputs = model(features)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy: {100 * correct / total:.2f}%')

Epoch [1/20], Loss: 1.9861888488133748
Validation Accuracy: 0.00%
Epoch [2/20], Loss: 1.9228874047597249
Validation Accuracy: 47.62%
Epoch [3/20], Loss: 1.8597843249638875
Validation Accuracy: 57.14%
Epoch [4/20], Loss: 1.8092201153437297
Validation Accuracy: 61.90%
Epoch [5/20], Loss: 1.740549882253011
Validation Accuracy: 61.90%
Epoch [6/20], Loss: 1.6732604106267293
Validation Accuracy: 71.43%
Epoch [7/20], Loss: 1.6068556308746338
Validation Accuracy: 71.43%
Epoch [8/20], Loss: 1.5402040878931682
Validation Accuracy: 80.95%
Epoch [9/20], Loss: 1.453330159187317
Validation Accuracy: 80.95%
Epoch [10/20], Loss: 1.3861066500345867
Validation Accuracy: 80.95%
Epoch [11/20], Loss: 1.3281381924947102
Validation Accuracy: 80.95%
Epoch [12/20], Loss: 1.2460627555847168
Validation Accuracy: 80.95%
Epoch [13/20], Loss: 1.1839660008748372
Validation Accuracy: 80.95%
Epoch [14/20], Loss: 1.0986272096633911
Validation Accuracy: 80.95%
Epoch [15/20], Loss: 1.0428913434346516
Validation Accuracy: